### Task for Data Quality Engineer

#### Data Quality Checks
This notebook performs data validation on customer, product, and transaction datasets.

In [1]:
import pandas as pd

In [2]:
customer_df = pd.read_csv("C:/Users/IntelliBoard/Downloads/Task for Data Quality Engineer/Task for Data Quality Engineer/Data/customers.csv", sep=';')
products_df = pd.read_csv("C:/Users/IntelliBoard/Downloads/Task for Data Quality Engineer/Task for Data Quality Engineer/Data/products.csv", sep=';')
transactions_df = pd.read_csv("C:/Users/IntelliBoard/Downloads/Task for Data Quality Engineer/Task for Data Quality Engineer/Data/transactions.csv", sep=';')

In [3]:
customer_df.head(10)

,customer_id,first_name,last_name,email
0,1,James,Martinez,david.brown@email.com
1,2,Jane,Lee,anna.miller@email.com
2,3,Sophie,Smith,sophie.brown@email.com
3,4,Robert,Lee,robert.williams@email.com
4,5,David,Smith,john.miller@email.com
5,6,Jane,Smith,jane.lee@email.com
6,7,Lisa,Smith,david.doe@email.com
7,8,Lisa,Johnson,robert.smith@email.com
8,9,John,Brown,robert.johnson@email.com
9,10,John,Taylor,jane.johnson@email.com


In [4]:
products_df.head(10)

,product_id,product_name,category,price
0,101,Product 101,Electronics,182.474044
1,102,Product 102,Appliances,112.870046
2,103,Product 103,Electronics,453.002773
3,104,Product 104,Electronics,225.062933
4,105,Product 105,Appliances,285.062664
5,106,Product 106,Appliances,485.428209
6,107,Product 107,Electronics,205.695362
7,108,Product 108,Clothing,388.884465
8,109,Product 109,Appliances,224.001381
9,110,Product 110,Electronics,427.471787


In [5]:
transactions_df.head(10)

,transaction_id,transaction_date,product_id,amount
0,1,2024-07-06,103,100.045934
1,2,2024-09-08,106,317.079944
2,3,2024-07-31,105,309.191028
3,4,2024-04-10,103,424.357274
4,5,2024-07-09,101,138.145672
5,6,2024-04-25,103,414.536235
6,7,NaN,104,328.273000
7,8,2024-08-30,106,344.829897
8,9,2024-10-03,106,126.410021
9,10,2024-04-06,106,404.427439


In [6]:


import re

# Load CSV files
customers= pd.read_csv("C:/Users/IntelliBoard/Downloads/Task for Data Quality Engineer/Task for Data Quality Engineer/Data/customers.csv", sep=';')
products = pd.read_csv("C:/Users/IntelliBoard/Downloads/Task for Data Quality Engineer/Task for Data Quality Engineer/Data/products.csv", sep=';')
transactions = pd.read_csv("C:/Users/IntelliBoard/Downloads/Task for Data Quality Engineer/Task for Data Quality Engineer/Data/transactions.csv", sep=';')



# Store tables in a dictionary
tables = {
    "transactions": transactions,
    "products": products,
    "customers": customers
}



# Create an empty list to store the report
report = []

def add_report(table, check, status, comment):
    row = {
        "Table": table,
        "Check": check,
        "Status": status,
        "Comment": comment
    }
    report.append(row)


    
# Check for missing values
def check_nulls(table_name, data_in_table):
    null_counts = data_in_table.isnull().sum()
    for column, count in null_counts.items():
        if count > 0:
            add_report(table_name, f"Missing values in '{column}'", "FAIL", f"{count} null values")
        else:
            add_report(table_name, f"Missing values in '{column}'", "OK", "No nulls")


            

# Check for duplicate rows
def check_duplicates(table_name, data_in_table):
    duplicate_count = data_in_table.duplicated().sum()
    if duplicate_count > 0:
        add_report(table_name, "Duplicate rows", "FAIL", f"{duplicate_count} duplicates found")
    else:
        add_report(table_name, "Duplicate rows", "OK", "No duplicates")


        
        
# Check if date column has valid format
def check_date_format(table_name, data_in_table, column):
    invalid_count = 0

    for value in data_in_table[column]:
        try:
            pd.to_datetime(value)
        except:
            invalid_count += 1

    if invalid_count > 0:
        add_report(table_name, f"Date format in '{column}'", "FAIL", f"{invalid_count} invalid date values")
    else:
        add_report(table_name, f"Date format in '{column}'", "OK", "Valid format")



        
        
# Check if price and amount are greater than 0
def check_positive_values():
    if "price" in products.columns:
        count = products[products["price"] <= 0].shape[0]
        if count > 0:
            add_report("products", "price > 0", "FAIL", f"{count} non-positive prices")
        else:
            add_report("products", "price > 0", "OK", "All prices are positive")
    else:
        add_report("products", "price > 0", "SKIPPED", "Column not found")

    if "amount" in transactions.columns:
        count = transactions[transactions["amount"] <= 0].shape[0]
        if count > 0:
            add_report("transactions", "amount > 0", "FAIL", f"{count} non-positive amounts")
        else:
            add_report("transactions", "amount > 0", "OK", "All amounts are positive")
    else:
        add_report("transactions", "amount > 0", "SKIPPED", "Column not found")


        

        
# Check foreign keys: product_id and customer_id in transactions
def check_foreign_keys():
    # Check product_id exists in products
    if "product_id" in transactions.columns and "product_id" in products.columns:
        for_product_check = transactions["product_id"].isin(products["product_id"])
        if not for_product_check.all():
            missing_count = (~for_product_check).sum()
            add_report("transactions", "product_id in products", "FAIL", f"{missing_count} missing")
        else:
            add_report("transactions", "product_id in products", "OK", "All product_id values exist in products")
    else:
        add_report("transactions", "product_id in products", "SKIPPED", "Column not found")

    # Check customer_id exists in customers
    if "customer_id" in transactions.columns and "customer_id" in customers.columns:
        for_customer_check = transactions["customer_id"].isin(customers["customer_id"])
        if not for_customer_check.all():
            missing_count = (~for_customer_check).sum()
            add_report("transactions", "customer_id in customers", "FAIL", f"{missing_count} missing")
        else:
            add_report("transactions", "customer_id in customers", "OK", "All customer_id values exist in customers")
    else:
        add_report("transactions", "customer_id in customers", "SKIPPED", "Column not found")




        
        
# Check if email addresses have valid format
def check_emails():
    email_column = "email"
    if email_column in customers.columns:
        pattern = r"^[\w\.-]+@[\w\.-]+\.\w+$"
        invalid_emails = customers[~customers[email_column].str.match(pattern, na=False)]
        count = invalid_emails.shape[0]
        if count > 0:
            add_report("customers", "Valid email format", "FAIL", f"{count} invalid emails")
        else:
            add_report("customers", "Valid email format", "OK", "All emails valid")
    else:
        add_report("customers", "Valid email format", "SKIPPED", "Column not found")

        

        
        
 # Run all checks
for table_name, data_in_table in tables.items():
    check_nulls(table_name, data_in_table)
    check_duplicates(table_name, data_in_table)

check_date_format("transactions", transactions, "transaction_date")
check_positive_values()
check_foreign_keys()
check_emails()



# Save the report to a CSV file
report_df = pd.DataFrame(report)
report_df.to_csv("data_quality_report.csv", index=False)
print("Report saved as 'data_quality_report.csv'")
print(report_df)

Report saved as 'data_quality_report.csv'
           Table                                 Check   Status  \
0   transactions    Missing values in 'transaction_id'       OK   
1   transactions  Missing values in 'transaction_date'     FAIL   
2   transactions        Missing values in 'product_id'       OK   
3   transactions            Missing values in 'amount'       OK   
4   transactions                        Duplicate rows       OK   
5       products        Missing values in 'product_id'       OK   
6       products      Missing values in 'product_name'       OK   
7       products          Missing values in 'category'       OK   
8       products             Missing values in 'price'       OK   
9       products                        Duplicate rows       OK   
10     customers       Missing values in 'customer_id'       OK   
11     customers        Missing values in 'first_name'       OK   
12     customers         Missing values in 'last_name'       OK   
13     customers    